In [ ]:
import Load_IEMOCAP
import Load_EmoDB
import sys
sys.path.append('../../lib/')
import mymodels
import NN_learning_tools
from torch.utils.data import Dataset, DataLoader
import numpy as np
import collections
import psutil

In [ ]:
# データセットをロードする
iemocap = Load_IEMOCAP.create_dataset()
emodb = Load_EmoDB.create_dataset()

In [ ]:
def zero_pad(x, pad_len):
    ret = np.zeros(pad_len)
    ret[:len(x)] = x
    return ret

In [ ]:
iemocap_data = iemocap.datas.copy()
emodb_data = emodb.datas.copy()

In [ ]:
iemocap_data = [data[0] for data in iemocap_data]
emodb_data = [data[0] for data in emodb_data]

In [ ]:
def zero_pad_(datas, std_len=50000):
    ret = []
    for data in datas:
        n_segments = np.ceil(len(data) / std_len)
        pad_len = int(std_len * n_segments)
        zero_padded = zero_pad(data, pad_len)
        ret.append(zero_padded)
    
    return ret

In [ ]:
iemocap_data = zero_pad_(iemocap_data)
emodb_data = zero_pad_(emodb_data)

In [ ]:
def segment_data(data, std_len=50000):
    if len(data) % std_len != 0:
        raise Exception(f'invalid data length: {len(data)}')
    ret = []
    n_segments = len(data) // std_len
    for i in range(n_segments):
        ret.append(data[i * std_len : (i + 1) * std_len])
    
    return ret

In [ ]:
iemocap_data = [segment_data(data) for data in iemocap_data]

In [ ]:
emodb_data = [segment_data(data) for data in emodb_data]

In [ ]:
iemocap_datalen = [len(data) for data in iemocap_data]
emodb_datalen = [len(data) for data in emodb_data]

In [ ]:
#wavファイルからメルペクトログラムを生成するメソッド
def calculate_melsp(x, n_fft=1024, hop_length=128):
    stft = np.abs(librosa.stft(x, n_fft=n_fft, hop_length=hop_length))**2
    log_stft = librosa.power_to_db(stft)
    melsp = librosa.feature.melspectrogram(S=log_stft,n_mels=128)
    return melsp